# 2. Data Preprocessing

참고 : https://github.com/kh-kim/nlp_with_pytorch_examples/tree/master/chapter-04

In [1]:
import sys, fileinput,re
from nltk.tokenize import sent_tokenize

### (1) 문장 단위 분절

#### a. 원래 문장

In [1]:
file_kor = open("Kor_input.txt","r",encoding='utf-8')

In [2]:
file_kor.read()

'자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것 입니다. 이에따라 이전까지 큰 변화가 없었던 자연어처리 분야의 연구는 폭발적으로 늘어나기 시작하여, 곧 기계번역 시스템은 신경망에 의해 정복 당하였습니다. 또한, attention 기법의 고도화로 전이학습이 발전하면서, QA 문제도 사람보다 정확한 수준이 되었습니다.'

#### b. 수정 문장

In [4]:
for line in open("Kor_input.txt","r",encoding='utf-8'):
    if line.strip() != "":
        line = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', line.strip())
        sentences = sent_tokenize(line.strip())
        
        for s in sentences :
            if s!="":
                sys.stdout.write(s+"\n")

자연어처리는 인공지능의 한 줄기 입니다.
시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다.
문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것 입니다.
이에따라 이전까지 큰 변화가 없었던 자연어처리 분야의 연구는 폭발적으로 늘어나기 시작하여, 곧 기계번역 시스템은 신경망에 의해 정복 당하였습니다.
또한, attention 기법의 고도화로 전이학습이 발전하면서, QA 문제도 사람보다 정확한 수준이 되었습니다.


### (2) 영어 분절
- 쉼표, 마침표, 인용부호 등을 띄워 줘야!

#### a. 원래 문장

In [5]:
file_eng = open("Eng_input.txt","r")

In [6]:
file_eng.read()

"Natural language processing is one of biggest stream in artificial intelligence, and it becomes very popular after seq2seq's invention. However, in order to make a strong A.I., there are still many challenges remain. I believe that we can breakthrough these barriers to get strong artificial intelligence."

#### b. 수정 문장

In [7]:
from nltk.tokenize.moses import MosesTokenizer

In [8]:
t = MosesTokenizer()

for line in open("Eng_input.txt","r"):
    if line.strip() != "":
        tokens = t.tokenize(line.strip(), escape=False)
        sys.stdout.write(' '.join(tokens) + '\n')

Natural language processing is one of biggest stream in artificial intelligence , and it becomes very popular after seq2seq 's invention . However , in order to make a strong A.I . , there are still many challenges remain . I believe that we can breakthrough these barriers to get strong artificial intelligence .


### (3) TorchText

### * 3가지 case
- case 1 ) X : corpus & Y : class ( 텍스트 분류, 감성 분석 )
- case 2 ) X : corpus ( 언어 모델 )
- case 3 ) X : corpus & Y : corpus ( 기계번역, 요약, 질의응답 )

### case 1 ) X : corpus & Y : class ( 텍스트 분류, 감성 분석 )
- 코퍼스 & 레이블 읽기
- 한 줄에서, text & class가 탭('\t')으로 구분된 데이터의 입력을 받는 경우

In [9]:
from torchtext import data

In [10]:
class DataLoader(object):
    def __init__(self, train_fn, valid_fn, 
                 batch_size=64, device=-1,
                 max_vocab=999999, min_freq=1, 
                 use_eos=False, shuffle=True):
        super(DataLoader,self).__init__()
        
        # define field of input file ( consists of 2 fields(columns) )
        # X ( corpus )
        self.text = data.Field(use_vocab=True,
                               batch_first=True,
                               include_lengths=False,
                               eos_token='<EOS>' if use_eos else None)
        # Y ( class )
        self.label = data.Field(sequential=False,
                               use_vocab=True,
                               unk_token=None)
        
        # 2 columns will be delimited by TAB
        train, valid = data.TabularDataset.splits(path='',
                                                 train=train_fn, validation=valid_fn, format='tsv',
                                                  fields=[ ('label', self.label),
                                                         ('text',self.text)])
        
        # train & valid iterator
        self.train_iter, self.valid_iter = data.BucketIterator.splits((train,valid), 
                                                                      batch_size=batch_size,
                                                                     device='cuda:%d' % device if device>=0 else 'cpu',
                                                                     shuffle=shuffle,
                                                                     sort_key=lambda x : len(x.text),
                                                                     sort_within_batch=True)
        
        # make vocabulary for label & text
        # ( = making mapping table between words & indice )
        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size=max_vocab,min_freq=min_freq)

### case 2 ) X : corpus ( 언어 모델 )
- 코퍼스 읽기
- 한 라인이 text로만 이루어진 경우 ( 주로 언어모델 훈련시킬 때 )
- 언어 모델(Language Model, LM) : 언어라는 현상을 모델링하고자 단어 시퀀스(또는 문장)에 확률을 할당(assign)하는 모델

In [11]:
from torchtext import data,datasets

In [12]:
PAD, BOS, EOS  = 1,2,3

In [13]:
class DataLoader():
    def __init__(self,train_fn,valid_fn,
                 batch_size=64,device='cpu',
                 max_vocab=99999999, max_length=255,
                 fix_length=None,use_bos=True,use_eos=True,
                 shuffle=True):
        super(DataLoader,self).__init__()
        
        # X (corpus)
        self.text = data.Field(sequential=True,
                              use_vocab=True,
                              batch_first=True,
                               include_lengths=True,
                               fix_length=fix_length,
                               init_token='<BOS>' if use_bos else None,
                               eos_token='<EOS>' if use_eos else None)
        
        train = LanguageModelDataset(path=train_fn,
                                    fields=[('text', self,text)],
                                    max_length=max_length)
        valid = LanguageModelDataset(path=valid_fn,
                                    fields=[('text', self.text)],
                                    max_length=max_length)
        
        self.train_iter = data.BucketIterator(train,
                                             batch_size=batch_size,
                                             device='cuda:%d' % device if device >= 0 else 'cpu',
                                             shuffle=False,
                                             sort_key = lambda x: -len(x.text),
                                             sort_within_batch=True)
        self.text.build_vocab(train, max_size=max_vocab)

In [14]:
class LanguageModelDataset(data.Dataset):
    def __init__(self, path, fields, max_length=None, **kwargs):
        if not isinstance(fields[0], (tuple,list)):
            fields = [('text', fields[0])]
            
        examples = []
        with open(path) as f :
            for line in f:
                line = line.strip()
                if max_length and max_length < len(line.split()):
                    continue
                if line != '':
                    examples.append(data.Example.fromlist([line],fields))
            
        super(LangaugeModelDataset, self).__init__(examples, fields, **kwargs)